# A Solution Approach to Crime Classification

**Author**: Jhon Adrián Cerón-Guzmán.<br>
**Date**: March 2020.<br>
**Description**: This is my solution approach to the [San Francisco Crime Classification](https://www.kaggle.com/c/sf-crime/) challenge proposed on Kaggle.

# 0. Requirements

In order to encourage reproducibility, the following is a list of technologies used, as well as their respective version:

1. Python **3.7.2**.
2. NumPy **1.17.2**.
3. SciPy **1.3.1**.
4. Scikit-learn **0.21.3**.
5. pandas **0.25.1**.
6. Matplotlib **3.1.1**.
7. Numba **0.48.0**.

In [ ]:
import calendar
import copy
import datetime
import os
import re

import numba
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [ ]:
CURRENT_PATH = os.path.abspath(os.getcwd())
DATA_PATH = os.path.join(CURRENT_PATH, 'data')

DATE_FORMAT = '%Y-%m-%d %H:%M:%S'
RANDOM_STATE = 91

ALGORITHMS = {
    'logit': {
        'estimator': LogisticRegression(),
        'predict_proba': True,
        'param_grid': {
            'solver': ['lbfgs'],
            'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
            }
        },
    'gb': {
        'estimator': GradientBoostingClassifier(),
        'predict_proba': True,
        'param_grid': None
        },
    'rf': {
        'estimator': RandomForestClassifier(),
        'predict_proba': True,
        'param_grid': None
        }
    }

In [ ]:
FNAMES = {
    'original-train': 'train.csv',
    'train': 'projected-train.csv',
    'original-test': 'test.csv',
    'test': 'projected-test.csv',
    'sample-submission': 'sampleSubmission.csv',
    'crime-dataset': 'crime-dataset.csv',
    'baseline': 'baseline-models.csv'
    }

FNAMES = {key: os.path.join(DATA_PATH, fname) for key, fname in FNAMES.items()}

# 1. Data Preprocessing

Let's map each crime category to its numerical representation.

In [ ]:
with open(FNAMES['sample-submission']) as f:
    for i, row in enumerate(f):
        row = row.rstrip('\n')
        CRIME_CATEGORY = {category: j-1 for j, category in enumerate(row.split(',')) if j > 0}
        
        break

In [ ]:
PROJECTION = ['Id', 'Dates', 'Category', 'DayOfWeek', 'PdDistrict', 'X', 'Y']

As specified by the variable `PROJECTION`, let's project (or filter) the datasets.

In [ ]:
def dataset_projection(in_fname, out_fname, projection=PROJECTION):
    valid_columns = []
    
    insert_id = False
    header, data = [], []
    with open(in_fname) as f:
        for i, row in enumerate(f):
            row = row.rstrip('\n')
            
            if i == 0:
                for j, col in enumerate(row.split(',')):
                    if col in projection:
                        header.append(col)
                        valid_columns.append(j)
                        
                insert_id = True if 'Id' not in header else False

                continue
                
            for old in re.findall(r'"[^"]+"', row):
                new = re.sub(r',', '|', old)                
                row = row.replace(old, new, 1)
                
            record = [
                re.sub(r'\|', ',', col).strip()
                for j, col in enumerate(row.split(',')) if j in valid_columns
                ]
            
            if len(record) != len(valid_columns):
                print('({}), Malformed columns at line {}'.format(in_fname, i+1))
                continue
            elif insert_id:
                record.insert(0, i-1)
                
            data.append(record)
            
    if insert_id:
        header.insert(0, 'Id')
    
    return header, data

In [ ]:
datasets = [
    [FNAMES['original-train'], FNAMES['train']],
    [FNAMES['original-test'], FNAMES['test']]
    ]
for in_fname, out_fname in datasets:
    if os.path.isfile(out_fname):
        continue
    
    header, data = dataset_projection(in_fname, out_fname)
    df = pd.DataFrame(data, columns=header)
    
    df['Dates'] = pd.to_datetime(df['Dates'], format=DATE_FORMAT)
    df = df.sort_values(by=['Dates'])
    df['Dates'] = df['Dates'].dt.strftime(DATE_FORMAT)
    
    df.to_csv(out_fname, index=False)

Finally, let's append the test set to the training one.

In [ ]:
if not os.path.isfile(FNAMES['crime-dataset']):
    columns = copy.deepcopy(PROJECTION)
    columns.remove('Category')
    columns.insert(0, 'Dataset')
    
    df = None
    for in_fname, dataset_type in [[FNAMES['train'], 'train'], [FNAMES['test'], 'test']]:
        dataset = pd.read_csv(in_fname)
        dataset['Dataset'] = dataset_type
        dataset = dataset[columns]
        
        df = dataset.copy(deep=True) if df is None else df.append(dataset)
        
    df['Dates'] = pd.to_datetime(df['Dates'], format=DATE_FORMAT)
    df = df.sort_values(by=['Dates', 'Dataset', 'Id'])
    df['Dates'] = df['Dates'].dt.strftime(DATE_FORMAT)
    
    df = df[columns]
    
    df.to_csv(FNAMES['crime-dataset'], index=False)

# 2. Feature Engineering

In [ ]:
@numba.jit(nopython=True)
def compute_distance(
        lat_1, lon_1,
        lat_2, lon_2):
    """Compute distance between two locations.
    
    Returns
    -------
    float
        Distance in KM.
    
    Source: <https://stackoverflow.com/questions/19412462/>
    """
    # Approximate radius of earth in KM
    earth_radius = 6373.0
    
    lat_1 = np.radians(lat_1)
    lon_1 = np.radians(lon_1)
    
    lat_2 = np.radians(lat_2)
    lon_2 = np.radians(lon_2)
    
    lon_dist = lon_2 - lon_1
    lat_dist = lat_2 - lat_1
    
    a = (np.square(np.sin(lat_dist/2))
         + np.cos(lat_1)
         * np.cos(lat_2)
         * np.square(np.sin(lon_dist/2)))
    
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    return earth_radius * c

In [ ]:
@numba.jit(nopython=True)
def compute_aggregated_features(data, time_window, crime_radius):
    """Compute aggregated features.
    
    Parameters
    ----------
    data : np.ndarray, dtype('int64')
        A Numpy-like array of shape "(n, m)", where "n" is the number
        of records and "m" is the number of columns (or attributes).
        The strict order of the columns is presented below:
            Dataset,
            Id,
            Dates,
            PdDistrict,
            X - Longitude,
            Y - Latitude
    time_window : int
        Time window (in hours).
    crime_radius : list
        List of integers, each of which representing a radius in kilometers.
    """
    n = len(data)
    
    # Let's transform the time window into seconds
    time_window = time_window * 60 * 60
    
    aggregated_features = []
    for i in range(n):
        ts = data[i,2]    
        
        lower_ts = ts - time_window
        
        mask = ((lower_ts < data[:,2])
                & (data[:,2] < ts))
        
        historical_data = data[mask]
        m = len(historical_data)
        
        police_district = data[i,3]
        
        feature_vector = [
            int(data[i,0]),
            int(data[i,1]),
            m, # number of crimes within the time window
            0 # number of crimes attended by the same police department district
            ]
        feature_vector = feature_vector + [0 for j in crime_radius]
        
        lat_1 = data[i,5]
        lon_1 = data[i,4]
        
        for j in range(m):
            feature_vector[3] += 1 if police_district == historical_data[j,3] else 0
            
            lat_2 = historical_data[j,5]
            lon_2 = historical_data[j,4]
            
            # Let's compute the number of crimes within each given radius
            distance = compute_distance(lat_1, lon_1, lat_2, lon_2)
            
            for k, rad in enumerate(crime_radius):
                feature_vector[4+k] += 1 if distance <= rad else 0
                
        aggregated_features.append(feature_vector)
        
    return aggregated_features

In [ ]:
def derive_date_attributes(df, column, drop_column=True):
    df['Year'] = df[column].dt.year
    
    df['Month'] = df[column].dt.month
    df['Quarter'] = df[column].dt.quarter
    
    df['Triannual'] = 0
    for i, (min_m, max_m) in enumerate([[1, 4], [5, 8], [9, 12]]):
        df.loc[((min_m <= df['Month']) & (df['Month'] <= max_m)), 'Triannual'] = i + 1
    
    df['Semester'] = 1
    df.loc[(df['Quarter'] > 2), 'Semester'] = 2
    
    df['Day'] = df[column].dt.day
    df['DayOfWeek'] = df[column].dt.dayofweek
    
    df['WorkingDay'] = 1
    df.loc[(df['DayOfWeek'] > 4), 'WorkingDay'] = 0
    
    df['Fortnight'] = 1
    df.loc[(df['Day'] > 15), 'Fortnight'] = 2
    
    df['Hour'] = df[column].dt.hour
    
    hourly_periods = {
        'four': [[i, i+3] for i in range(0, 24, 4)],
        'six': [[i, i+5] for i in range(0, 24, 6)],
        'twelve': [[i, i+11] for i in range(0, 24, 12)],
        }
    
    for str_period, period in hourly_periods.items():
        period_column = '{}HourPeriod'.format(str_period.title())
        df[period_column] = 0
        
        for i, (min_hr, max_hr) in enumerate(period):
            df.loc[((min_hr <= df['Hour']) & (df['Hour'] <= max_hr)), period_column] = i + 1
    
    if drop_column:
        df = df.drop(columns=[column])
    
    return df

In [ ]:
def feature_engineering(
        df, data, time_windows,
        idx_to_dataset, idx_to_district,
        crime_radius=[1, 2, 4, 8, 16]):
    """Compute the process of feature engineering."""
    df = df[['Dataset', 'Id', 'PdDistrict', 'Dates']]
    df = df.replace({'Dataset': idx_to_dataset, 'PdDistrict': idx_to_district})
    
    df = derive_date_attributes(df, 'Dates')
    
    crime_radius = np.array(crime_radius, dtype=int)
    
    agg_ds_fname = os.path.join(DATA_PATH, 'agg-dataset-{}H.csv')
    cum_ds_fname = os.path.join(DATA_PATH, 'agg-dataset-{}H-cumulative.csv')    
    
    for i, time_window in enumerate(time_windows):
        agg_ds_fname_1 = agg_ds_fname.format(time_window)
        cum_ds_fname_1 = cum_ds_fname.format(time_window)
        
        if (os.path.isfile(cum_ds_fname_1)
                or (i == 0 and os.path.isfile(agg_ds_fname_1))):
            continue
        elif os.path.isfile(agg_ds_fname_1):
            agg_ds = pd.read_csv(agg_ds_fname_1)
            
            for col in agg_ds.columns:
                if col in ['Dataset']:
                    continue
                
                agg_ds[col] = pd.to_numeric(agg_ds[col])
        else:
            agg_ds = compute_aggregated_features(data, time_window, crime_radius)
            
            prefix = '{}H_'.format(time_window)
            agg_ds_columns = (['Dataset', 'Id']
                              + [(prefix + col) for col in ['Crimes', 'CrimesAttendedByPdDistrict']]
                              + [(prefix + 'CrimesWithin{}KMRad'.format(rad)) for rad in crime_radius])
            
            agg_ds = pd.DataFrame(agg_ds, columns=agg_ds_columns)
            agg_ds = agg_ds.astype({col: 'int32' for col in agg_ds_columns})
            
            agg_ds['Dataset'] = agg_ds['Dataset'].map(idx_to_dataset)
            
            agg_ds.to_csv(agg_ds_fname_1, index=False)
        
        if i == 0:
            continue
            
        cum_ds = agg_ds.copy(deep=True)
        agg_ds = None
        
        for j in range(i):
            agg_ds_fname_2 = agg_ds_fname.format(time_windows[j])
            
            agg_ds = pd.read_csv(agg_ds_fname_2)
            
            for col in agg_ds.columns:
                if col in ['Dataset']:
                    continue
                
                agg_ds[col] = pd.to_numeric(agg_ds[col])
                
            cum_ds = pd.merge(agg_ds, cum_ds, on=['Dataset', 'Id'], how='inner')
            
        cum_ds = pd.merge(df, cum_ds, on=['Dataset', 'Id'], how='inner')
        
        cum_ds.to_csv(cum_ds_fname_1, index=False)

In [ ]:
df = pd.read_csv(FNAMES['crime-dataset'])

df['Dates'] = pd.to_datetime(df['Dates'], format=DATE_FORMAT)

In [ ]:
DATASET_TO_IDX = {dataset: i for i, dataset in enumerate(df['Dataset'].unique())}
IDX_TO_DATASET = {i: dataset for dataset, i in DATASET_TO_IDX.items()}

df['Dataset'] = df['Dataset'].map(DATASET_TO_IDX)

In [ ]:
DISTRICT_TO_IDX = {district: i for i, district in enumerate(df['PdDistrict'].unique())}
IDX_TO_DISTRICT = {i: district for district, i in DISTRICT_TO_IDX.items()}

df['PdDistrict'] = df['PdDistrict'].map(DISTRICT_TO_IDX)

In [ ]:
PROJECTION = (['Dataset']
              + [col for col in PROJECTION if col not in ['Category', 'DayOfWeek']])

df = df[PROJECTION]
df = df.sort_values(by=['Dates', 'Dataset', 'Id'])

crimes = df.copy(deep=True)
crimes['ts'] = crimes['Dates'].values.astype(np.int64) // 10 ** 9
crimes = crimes[[('ts' if col == 'Dates' else col) for col in PROJECTION]].to_numpy().astype(float)

Bear in mind that running the feature engineering process takes a long time; approximately 7.3 hours.

In [ ]:
TIME_WINDOWS = [12, 24, 72, 168, 336]

feature_engineering(df, crimes, TIME_WINDOWS, IDX_TO_DATASET, IDX_TO_DISTRICT)

# 3. Crime Classification

## 3.1 Train-Test Split

Let's split the whole dataset of crimes into training, validation and test datasets. Please recall that the original training and test datasets, as downloaded from Kaggle, were merged to run the feature engineering process. Lastly, the training dataset will be split into two folds: the first one, approximately 80% of the data, is intended to train prediction models; the second one is aimed at validating each prediction model on an independent dataset.

In [ ]:
VALIDATION_SIZE = 0.2

for time_window in TIME_WINDOWS:
    in_fname = os.path.join(DATA_PATH, 'agg-dataset-{}H-cumulative.csv'.format(time_window))
    if not os.path.isfile(in_fname):
        continue
    
    window_path = {
        '/': os.path.join(DATA_PATH, '{}H'.format(time_window))
        }
    window_path['data'] = os.path.join(window_path['/'], 'data')
    
    for path in window_path.values():
        if not os.path.isdir(path):
            os.makedirs(path)
    
    window_fnames = {
        ds: os.path.join(window_path['data'], '{}.csv'.format(ds))
        for ds in ['train', 'validation', 'test']
        }
    
    split_data = False
    for fname in window_fnames.values():
        if not os.path.isfile(fname):
            split_data = True
            break
            
    if not split_data:
        continue
    
    df = pd.read_csv(in_fname)    
    
    df['Id'] = pd.to_numeric(df['Id'])
    
    columns = df.columns    
    
    numerical_columns = [col for col in columns if re.match(r'[0-9]{2,3}H_', col)]    
    
    categorical_columns = ['PdDistrict']
    for col in columns:
        if (col not in numerical_columns
                and col not in ['Dataset', 'Id', 'PdDistrict']):
            categorical_columns.append(col)
    
    if not os.path.isfile(window_fnames['test']):
        test = df.loc[df['Dataset']=='test'].drop(columns=['Dataset'])
        
        test = test.sort_values(by=['Id'])
        
        columns = ['Id'] + categorical_columns + numerical_columns
        test = test[columns]
        
        test.to_csv(window_fnames['test'], index=False)
        test = None
        
    df = df.loc[df['Dataset']=='train'].drop(columns=['Dataset'])
    
    train = pd.read_csv(FNAMES['train'])    
    
    train['Id'] = pd.to_numeric(train['Id'])
    train = train[['Id', 'Category']]
    
    assert len(df) == len(train)
    
    df = pd.merge(df, train, on=['Id'], how='inner')
    train = None
    
    columns = ['Id'] + categorical_columns + numerical_columns + ['Category']
    df = df[columns]
    
    # The validation dataset is a stratified random sample
    
    validation = None
    for category in CRIME_CATEGORY.keys():
        category_samples = df.loc[df['Category']==category]
        
        sample_size = len(category_samples) * VALIDATION_SIZE
        sample_size = int(np.round(sample_size, 0))
        
        sample = category_samples.sample(n=sample_size, replace=False, random_state=RANDOM_STATE)
        
        validation = (
            validation.append(sample).reset_index(drop=True)
            if validation is not None
            else sample.copy(deep=True)
            )
    
    validation.to_csv(window_fnames['validation'], index=False)
    
    df = df.loc[~df['Id'].isin(validation['Id'].values)]
    df.to_csv(window_fnames['train'], index=False)

## 3.2 Baseline Model

Let's build a strong baseline model according to the following criteria:

1. The training dataset will be used to learn prediction models.
2. The validation dataset will be used to rank prediction models, as they haven't seen these data during their training process.
3. The entire set of features will be used.
4. A set of several machine learning algorithms will be used to learn prediction models. Thus, a prediction model will be learned per each combination of algorithm and time window.
5. There will not be hyperparameter optimization. Machine learning algorithms will be used with their default hyperparameters.

In [ ]:
def identify_attributes(
        df, attributes, time_window, window_type,
        attributes_to_exclude=['Id', 'PdDistrict', 'Category']
        ):
    """Identify the attribute names the sets of numerical and categorical attributes consist of.
    
    Returns
    -------
    list
        List of attribute names the set of numerical features consists of.
    list
        List of attribute names the set of categorical features consists of.
    """
    columns = df.columns
    
    num_attr_re = re.compile(
        r'{}H_'.format('[0-9]{2,3}' if window_type == 'cumulative' else time_window)
        )
    num_attr = (
        [col for col in columns if num_attr_re.match(col)]
        if attributes in ['num', 'all']
        else []
        )
    
    cat_attr = []
    for col in columns:
        if attributes == 'num':
            break
        
        if (col not in attributes_to_exclude
                and not re.match(r'[0-9]{2,3}H_', col)):
            cat_attr.append(col)
            
    return num_attr, cat_attr

In [ ]:
def build_model(
        train, validation,
        attributes, y_column,
        time_window, window_type,
        estimator, param_grid,
        predict_proba,
        return_pred=False
        ):
    """Build a prediction model.
    
    Parameters
    ----------
    train : pd.DataFrame
    
    validation : pd.DataFrame
    
    attributes : str
        The set of attributes to be used as input by the estimator.
        
        - If "cat", then the date-derived attributes are used.
        - If "num", then the aggregated attributes computed using the time window are used.
        - If "all", then the union of the attributes "cat" and "raw" is used.
        
        Bear in mind that, whatever the set of attributes,
        the attribute "PdDistrict" is always used.
    
    y_column : str
        Which is the target variable? This variable must
        be in both training and validation datasets.
    
    time_window : int
        Time window (in hours).
    
    window_type : str
        Whether to use other windows whose time in hours is less than the specified time window value.
        
        - If "exact", then aggregated attributes that were computed using only the specified time
          window, are used.
        - If "cumulative", then aggregated attributes that were computed using windows whose time
          in hours is less than the specified time window, are also used.
    
    estimator
        A scikit-learn classification estimator.
    
    param_grid : dict
        Dictionary of parameters, as well as their corresponding values, for the estimator.
        This enables an exhaustive search over the specified parameter values through cross-validation.
        
    predict_proba : bool
        Whether or not the estimator supports the "predict_proba()" method.
    
    return_pred : bool
    """
    num_attr, cat_attr = identify_attributes(train, attributes, time_window, window_type)
    
    if 'PdDistrict' in train.columns:
        cat_attr.insert(0, 'PdDistrict')
    
    X_train_cat = train[cat_attr].to_numpy()
    X_valid_cat = validation[cat_attr].to_numpy()
    
    X_train_num = None
    X_valid_num = None
    
    if attributes in ['num', 'all']:
        scaler = StandardScaler()
        X_train_num = scaler.fit_transform(train[num_attr].to_numpy().astype(float))
        
        X_valid_num = scaler.transform(validation[num_attr].to_numpy().astype(float))
    
    X_train = (
        np.hstack([X_train_cat, X_train_num])
        if X_train_num is not None
        else X_train_cat
        )
    
    X_valid = (
        np.hstack([X_valid_cat, X_valid_num])
        if X_valid_num is not None
        else X_valid_cat
        )
    
    y_train = train[y_column].values.astype(int)
    y_valid = validation[y_column].values.astype(int)
    
    clf = estimator
    
    if isinstance(param_grid, dict):
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
        
        scoring = 'neg_log_loss' if predict_proba else 'f1_macro'
        clf = GridSearchCV(
            estimator=estimator, param_grid=param_grid, cv=cv,
            n_jobs=6, scoring=scoring, iid=False, refit=True
            )
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict_proba(X_valid) if predict_proba else clf.predict(X_valid)    
    
    score = log_loss(y_valid, y_pred)
    
    if not return_pred:
        return score
    
    return score, y_pred

In [ ]:
def find_baseline_model(
        time_windows, out_fname,
        district_to_idx=DISTRICT_TO_IDX,
        crime_category=CRIME_CATEGORY,
        algorithms=ALGORITHMS):
    """Find the best baseline model, as specified above."""
    results = []
    results_header = ['TimeWindow', 'Algorithm', 'LogLoss']
    
    for time_window in time_windows:
        window_path = {
            '/': os.path.join(DATA_PATH, '{}H'.format(time_window))
            }
    
        window_path['data'] = os.path.join(window_path['/'], 'data')
        
        train = pd.read_csv(os.path.join(window_path['data'], 'train.csv'))
        train = train.replace({'PdDistrict': district_to_idx, 'Category': crime_category})
        
        validation = pd.read_csv(os.path.join(window_path['data'], 'validation.csv'))
        validation = validation.replace({'PdDistrict': district_to_idx, 'Category': crime_category})
        
        for algo, settings in algorithms.items():
            score = build_model(
                train, validation,
                'all', 'Category',
                time_window, 'cumulative',
                settings['estimator'], None,
                settings['predict_proba']
                )
            results.append([str(time_window), algo, '{:.5f}'.format(score)])
            
            pd.DataFrame(results, columns=results_header).to_csv(out_fname, index=False, mode='w')

Finding a strong baseline model takes a long time. Therefore, the set of time windows will be reduced, namely: 24, 72, and 168 hours.

In [ ]:
TIME_WINDOWS.pop(0)

if not os.path.isfile(FNAMES['baseline']):
    find_baseline_model(TIME_WINDOWS[:-1], FNAMES['baseline'])

Let's analyze these results.

In [ ]:
baseline_results = pd.read_csv(FNAMES['baseline'])

for col in baseline_results.columns:
    if col in ['Algorithm']:
        continue
    
    baseline_results[col] = pd.to_numeric(baseline_results[col])

In [ ]:
chart_data = baseline_results.pivot(index='TimeWindow', columns='Algorithm', values='LogLoss')

print(chart_data)

chart = chart_data.plot(kind='bar', grid=True, legend=True, x=None)
chart.set_xlabel('Time window')
chart.set_ylabel('Log loss')

Notably, the Logistic Regression algorithm outperforms all of the machine learning algorithms, even though the best result, i.e., **2.58249**, was achieved by the Gradient Boosting algorithm on data aggregated using a time window of 72 hours. This conclusion is drawn as Logistic Regression is the most computationally efficient algorithm, and the difference between its best result and the overall best result is negligible.

On the other hand, let's analyze how having a larger time window contributes to crime classification. The Logistic Regression algorithm will be used to perform this analysis.

In [ ]:
FNAMES['logit-baseline'] = os.path.join(DATA_PATH, 'baseline-logit-models.csv')

if not os.path.isfile(FNAMES['logit-baseline']):
    logit_baseline = baseline_results.loc[baseline_results['Algorithm']=='logit']
    logit_baseline = logit_baseline.drop(columns=['Algorithm'])
    
    time_window = 336
    
    window_data_path = os.path.join(DATA_PATH, '{}H'.format(time_window), 'data')
    
    train = pd.read_csv(os.path.join(window_data_path, 'train.csv'))
    train = train.replace({'PdDistrict': DISTRICT_TO_IDX, 'Category': CRIME_CATEGORY})
    
    validation = pd.read_csv(os.path.join(window_path['data'], 'validation.csv'))
    validation = validation.replace({'PdDistrict': DISTRICT_TO_IDX, 'Category': CRIME_CATEGORY})
    
    algo = ALGORITHMS['logit']
    
    score = build_model(
        train, validation,
        'all', 'Category',
        time_window, 'cumulative',
        algo['estimator'], None,
        algo['predict_proba']
        )
    
    logit_baseline = logit_baseline.append({'TimeWindow': time_window, 'LogLoss': score}, ignore_index=True)
    
    logit_baseline.to_csv(FNAMES['logit-baseline'], index=False, mode='w')

In [ ]:
logit_baseline = pd.read_csv(FNAMES['logit-baseline'])

logit_baseline['LogLoss'] = pd.to_numeric(logit_baseline['LogLoss'])

print(logit_baseline)

chart = logit_baseline.plot(x='TimeWindow', y='LogLoss', kind='bar', grid=True, legend=False)
chart.set_xlabel('Time window')
chart.set_ylabel('Log loss')

To sum up, the **baseline result** is **2.5947**.

## 3.3 Discriminative Features

The goal is to quantify how discriminative each set of features is, as well as each window type.

In [ ]:
def discriminative_feature(
        time_window, algorithm, out_fname,
        district_to_idx=DISTRICT_TO_IDX,
        crime_category=CRIME_CATEGORY):
    """Quantify how discriminative each set of features is, as well as each window type."""
    window_data_path = os.path.join(DATA_PATH, '{}H'.format(time_window), 'data')
    
    train = pd.read_csv(os.path.join(window_data_path, 'train.csv'))
    train = train.replace({'PdDistrict': district_to_idx, 'Category': crime_category})
    
    validation = pd.read_csv(os.path.join(window_data_path, 'validation.csv'))
    validation = validation.replace({'PdDistrict': district_to_idx, 'Category': crime_category})
    
    grid = {
        'attributes': ['all', 'num', 'cat'],
        'window_type': ['exact', 'cumulative']
        }
    
    results = []
    results_header = ['Attributes', 'WindowType', 'LogLoss']
    
    for params in ParameterGrid(grid):
        score = build_model(
            train, validation,
            params['attributes'], 'Category',
            time_window, params['window_type'],
            algorithm['estimator'], None,
            algorithm['predict_proba']
            )
        
        results.append([params['attributes'], params['window_type'], '{:.5f}'.format(score)])
        
        pd.DataFrame(results, columns=results_header).to_csv(out_fname, index=False, mode='w')

In [ ]:
FNAMES['discriminative-features'] = os.path.join(DATA_PATH, 'discriminative-features.csv')

if not os.path.isfile(FNAMES['discriminative-features']):
    discriminative_feature(
        336, ALGORITHMS['logit'], FNAMES['discriminative-features']
        )

In [ ]:
discriminative_features = pd.read_csv(FNAMES['discriminative-features'])

discriminative_features['LogLoss'] = pd.to_numeric(discriminative_features['LogLoss'])

In [ ]:
chart_data = discriminative_features.pivot(index='Attributes', columns='WindowType', values='LogLoss')

print(chart_data)

chart = chart_data.plot(kind='bar', grid=True, legend=True)

chart.legend(title='Window type', bbox_to_anchor=(1.0, 1.0))

chart.set_xlabel('Attributes')
chart.set_ylabel('Log loss')

These results show that the union of the numerical and categorical attributes contribute the most to discriminative power. In a like manner, aggregated attributes computed using cumulative time windows rather than an exact time window are more discriminative.

## 3.4 Ensemble Method

In [ ]:
def write_in_file(fname, content, mode='w', insert_new_line=True):
    with open(fname, mode) as f:
        f.write(content + ('\n' if insert_new_line else ''))

In [ ]:
@numba.jit(nopython=True)
def is_leap_year(year):
    return (True
            if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))
            else False)

In [ ]:
@numba.jit(nopython=True)
def get_number_of_days_in_month(data):
    n = len(data)
    
    days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    result = []
    for i in range(n):
        year = data[i,0]
        month = data[i,1]
        
        result.append(
            (29 if is_leap_year(year) else 28)
            if month == 2
            else days_in_month[month-1]
            )
    
    return result

In [ ]:
def encode_cyclical_attributes(
        df,
        attributes=['Month', 'Day', 'DayOfWeek', 'Hour']):
    """Encoding of cyclical attributes.
    
    Source: <http://blog.davidkaleko.com/feature-engineering-cyclical-features.html>
    """
    df['days_month'] = get_number_of_days_in_month(df[['Year', 'Month']].to_numpy().astype(int))
    
    for attr in attributes:
        max_val = df[attr].max() if attr != 'Day' else df['days_month']
        min_val = df[attr].min()
        
        if min_val == 1:
            df[attr] -= 1
        elif min_val == 0:
            max_val += 1
        
        df['{}_Sin'.format(attr)] = np.sin(df[attr] * 2 * np.pi / max_val)
        df['{}_Cos'.format(attr)] = np.cos(df[attr] * 2 * np.pi / max_val)
        
    attributes.append('days_month')
    df = df.drop(columns=attributes)
    
    return df

In [ ]:
def encode_categorical_attributes(df, attributes):
    """One-Hot encoding of categorical attributes."""
    return pd.get_dummies(df, prefix=attributes, columns=attributes)

In [ ]:
def split_training_data(df, n_splits, split_criterion='Category'):
    criterion_values = {
        value: df.loc[df[split_criterion]==value].shape[0]
        for value in df[split_criterion].unique()
        }
    
    sampled_instances = []
    
    for i in range(n_splits):
        last_split = True if i == (n_splits-1) else False
        
        sample = None
        for value, size in criterion_values.items():
            mask = (df[split_criterion] == value) & (~df['Id'].isin(sampled_instances))
            criterion_sample = df.loc[mask]
            
            sample_size = size * (1/n_splits)
            sample_size = int(np.round(sample_size, 0))
            
            if not last_split:
                criterion_sample = criterion_sample.sample(
                    n=sample_size, replace=False, random_state=RANDOM_STATE
                    )
                
            sample = (
                sample.append(criterion_sample).reset_index(drop=True)
                if sample is not None
                else criterion_sample.copy(deep=True)
                )
            
            sampled_instances += criterion_sample['Id'].values.tolist()
        
        yield sample

In [ ]:
def build_classifier_ensembles(
        algorithm=ALGORITHMS['logit'],
        crime_category=CRIME_CATEGORY,
        district_to_idx=DISTRICT_TO_IDX,
        time_windows=TIME_WINDOWS):
    """Build classifier ensembles."""
    prediction_header = ['' for j in range(len(crime_category))]
    for crime, idx in crime_category.items():
        prediction_header[idx] = crime
    
    ensemble_grid = {
        'encode_cyclical_attr': [True, False],
        'encode_date_attr': [True, False],
        'encode_district_attr': [True, False],
        'n_estimators': [3, 5, 7],
        'stack_method': ['hard_voting', 'soft_voting', 'clf'],
        'time_window': time_windows
        }
    
    for settings in ParameterGrid(ensemble_grid):
        time_window = settings['time_window']
        
        window_path = {
            '/': os.path.join(DATA_PATH, '{}H'.format(time_window))
            }        
        window_path['/data'] = os.path.join(window_path['/'], 'data')
        
        train = pd.read_csv(os.path.join(window_path['/data'], 'train.csv'))
        validation = pd.read_csv(os.path.join(window_path['/data'], 'validation.csv'))
        
        if settings['encode_cyclical_attr']:
            train = encode_cyclical_attributes(train)
            validation = encode_cyclical_attributes(validation)
        
        if settings['encode_date_attr']:
            date_attr = [
                'Quarter', 'Triannual', 'Semester', 'Fortnight',
                'FourHourPeriod', 'SixHourPeriod', 'TwelveHourPeriod'
                ]
            
            train = encode_categorical_attributes(train, date_attr)
            validation = encode_categorical_attributes(validation, date_attr)
        
        if settings['encode_district_attr']:
            train = encode_categorical_attributes(train, ['PdDistrict'])
            validation = encode_categorical_attributes(validation, ['PdDistrict'])
        else:
            train = train.replace({'PdDistrict': district_to_idx})
            validation = validation.replace({'PdDistrict': district_to_idx})
            
        train = train.replace({'Category': crime_category})
        validation = validation.replace({'Category': crime_category})
        
        window_path['/prediction'] = os.path.join(
            window_path['/'],
            'prediction',
            'validation',
            'stack_method={}'.format(settings['stack_method']),
            'n_estimators={}'.format(settings['n_estimators']),
            'cyclical_attr={};date_attr={};district_attr={}'.format(*[
                ('True' if settings['encode_{}_attr'.format(var)] else 'False')
                for var in ['cyclical', 'date', 'district']
                ])
            )
        
        if not os.path.isdir(window_path['/prediction']):
            os.makedirs(window_path['/prediction'])
            
        learners_result_fname = os.path.join(window_path['/prediction'], 'learners-result.csv')
        if not os.path.isfile(learners_result_fname):
            write_in_file(learners_result_fname, ','.join(['Clf', 'LogLoss']))
        
        for i, train_sample in enumerate(split_training_data(train, settings['n_estimators'])):
            pred_fname = os.path.join(window_path['/prediction'], 'clf-{}-pred.csv'.format(i))
            if os.path.isfile(pred_fname):
                continue
            
            score, predictions = build_model(
                train_sample, validation,
                'all', 'Category',
                time_window, 'cumulative',
                algorithm['estimator'], None,
                algorithm['predict_proba'],
                return_pred=True
                )
            
            write_in_file(
                learners_result_fname, ','.join([str(i), '{:.5f}'.format(score)]), mode='a'
                )
            
            predictions = pd.DataFrame(predictions, columns=prediction_header)
            
            predictions['Id'] = validation['Id'].values
            predictions = predictions[['Id']+prediction_header]
            
            predictions.to_csv(pred_fname, float_format='%.5f', index=False)

In [ ]:
build_classifier_ensembles()